In [2]:
# Import of required modules
import pandas as pd
from statsmodels.tsa.ar_model import AR
from sklearn.model_selection import train_test_split

# Manually defined format of reading dates
dateparse = lambda dates: pd.datetime.strptime(dates, '%d/%m/%Y')

# Reading Purchase Order table from database, converting column containing date to datetime format recognised by Python
df1 = pd.read_csv('PurchaseOrder.2.csv', parse_dates=['Order_Date'], index_col='Order_Date',date_parser=dateparse)

# Reading Purchase Order Details and Catalogue table from database
df2 = pd.read_csv('PurchaseOrderDetails.2.csv')
df3 = pd.read_csv('Catalogue(Raw).csv')

df = df1.reset_index().merge(df2)

# Storing of unique item names
uniqueitem = df['Item_Num'].unique()

# Creation of empty array for predictions made by ML
predictions = {}

# For each item stored in line 9, to conduct ML process
for id in range(0, len(uniqueitem)):
    df2 = df.loc[df['Item_Num'] == uniqueitem[id]]
    ts = df2[['Order_Date','Quantity']]
    ts.Order_Date = pd.to_datetime(ts.Order_Date)
    ts.set_index('Order_Date', inplace=True)
    
    # Summarising ‘Quantity’ values on a monthly level
    ts = ts.resample('MS').sum()
    X = ts.values
    
    # Setting training size to utilise full 12 months’ worth of data
    train, test = train_test_split(X, test_size = 0, random_state = 0)
    model = AR(train)
    model_fit = model.fit()
    
    # Retrieving one-step-ahead forecast based on training data
    prediction = model_fit.predict(start=len(train), end=len(train)+len(test))
    
    # Saving predictions made by item name in a dictionary
    predictions.update( {uniqueitem[id] : prediction[0]} )
    
# Creating a column containing prediction values
df3['Predicted_Quantity'] = df3.Item_Num.map(predictions)

# Storing predicted values back into Catalogue table, to be inserted into database
df3.to_csv("Catalogue.csv", index=False)

,Item_Num,Category,Description,Reorder_Level,Reorder_Quantity,Predicted_Quantity,Available_Quantity,UOM,Price,Bin_Number,Supplier_1,Supplier_2,Supplier_3
0,C001,Clip,"Clips Double 1""",50,30,1258.666763,500,Dozen,20,NaN,NaN,NaN,NaN
1,C002,Clip,"Clips Double 2""",50,30,984.469947,500,Dozen,20,NaN,NaN,NaN,NaN
2,C003,Clip,"Clips Double 3/4""",50,30,1130.309504,500,Dozen,20,NaN,NaN,NaN,NaN
3,C004,Clip,Clips Paper Large,50,30,741.022588,500,Box,20,NaN,NaN,NaN,NaN
4,C005,Clip,Clips Paper Medium,50,30,1414.619570,500,Box,20,NaN,NaN,NaN,NaN
5,C006,Clip,Clips Paper Small,50,30,1044.033419,500,Box,20,NaN,NaN,NaN,NaN
6,E001,Envelope,"Envelope Brown (3""x6"")",600,400,943.718643,500,Each,20,NaN,NaN,NaN,NaN
7,E002,Envelope,"Envelope Brown (3""x6"") w/ Window",600,400,1521.414809,500,Each,20,NaN,NaN,NaN,NaN
8,E003,Envelope,"Envelope Brown (5""x7"")",600,400,969.760110,500,Each,20,NaN,NaN,NaN,NaN
9,E004,Envelope,"Envelope Brown (5""x7"") w/ Window",600,400,1115.073801,500,Each,20,NaN,NaN,NaN,NaN
